# Spotify API exploration

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
import config

this is how to set environment variables in a jupyter notebook

In [3]:
#%env MY_VAR=MY_VALUE

In [9]:
print(type(config.SPOTIFY_CLIENT_ID))

<class 'str'>


In [10]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.SPOTIFY_CLIENT_ID,
                                                           client_secret=config.SPOTIFY_SECRET))

results = sp.search(q='lil baby', limit=20)
for idx, track in enumerate(results['tracks']['items']):
    print(idx, track['name'])

0 Freestyle
1 Wants and Needs (feat. Lil Baby)
2 Drip Too Hard (Lil Baby & Gunna)
3 On Me
4 Forever (feat. Fridayy)
5 Baby (Lil Baby feat. DaBaby)
6 Pure Cocaine
7 Detox
8 Yes Indeed
9 EVERY CHANCE I GET (feat. Lil Baby & Lil Durk)
10 In A Minute
11 California Breeze
12 Sleazy Flow (with Lil Baby) - Remix
13 Never Hating (feat. Young Thug)
14 Heyy
15 Stand On It
16 Not Finished
17 STAYING ALIVE (feat. Drake & Lil Baby)
18 The World Is Yours To Take - Budweiser Anthem Of The FIFA World Cup 2022
19 Heyy


Get a playlist of mine  
can just retrieve playlist with the link

## idea here is to look at a playlist that the user provides

In [ ]:
results = sp.playlist('https://open.spotify.com/playlist/3LNpCyZZnzP0YQDLC52CKS?si=81c062b03e8447b1')

In [ ]:
print(results)

In [ ]:
results.keys()

In [ ]:
print(results['external_urls'])

In [ ]:
results['href']

In [ ]:
import json

In [ ]:
print(json.dumps(results, indent=2, default=str))

In [ ]:
print (json.dumps(results["tracks"]["items"], indent=2, default=str))

This the names of all of the tracks in the playlist

In [ ]:
res = results["tracks"]["items"]

playlist_df = pd.DataFrame(columns=['song_title', 'artist','artist_id' ,'album', 'track_id', 'popularity'], index=None)
track_position = 1
for i in res:
    track = i["track"]
#    print(i, end='')
#    print(i.keys())
#    print(track) 
    name = track["name"]
    #we should get track ID

    artists = []
    artist_list = track["artists"]
    for a in range(len(artist_list)):
        artists.append((artist_list[a]['name'], artist_list[a]['id']))
        #at this level we can grab artist ID as well
        
    
        
    
    album = track["album"]["name"]
    #should be able to grab album ID as well
    track_id = track["id"]
    artist_list = [x[0] for x in artists]
    artist_id_list = [x[1] for x in artists]
    
    #track popularity?
    track_pop = track["popularity"]
    

    
    print('name:', name)
    print('artist:', artists)
    print('album:', album)
    print('artist_ids:', artist_id_list)
    print('popularity', track_pop)
    print("-------------------------------")
    
#     artist_id_list = [x[1] for x in artists]
    
            
    playlist_df.loc[track_position] = {'song_title' : name,
                                   'artist' : artist_list,
                                   'artist_id' : artist_id_list,
                                   'album' : album, 
                                   'track_id' : track_id,
                                    'popularity':track_pop} #can i put a list into one column? 
    track_position+=1


In [ ]:
playlist_df

## Get audio features from songs on playlists

In [ ]:
#a list of all of the track ids in the playlist
tracklist = playlist_df.track_id.ravel()
tracklist
#tracklist[0]

In [ ]:
track_audio_features = sp.audio_analysis(tracklist[0])['track']

In [ ]:
#print(json.dumps(track_audio_features, indent=2, default=str))

In [ ]:
try:
    track_audio_features = sp.audio_features(tracklist)
    for feature in track_audio_features:
        print(json.dumps(feature, indent=4))
        print()
except AttributeError:
    null_song_idx = np.where(tracklist == None)
    pass

#drop songs that arent found in spotify
tracklist = [x for x in tracklist if (x != None)]

track_audio_features = sp.audio_features(tracklist)

# for feature in track_audio_features:
#     print(json.dumps(feature, indent=4))
#     print()
print('These are the songs which could not be analyzed. (this is probably because theyre not on spotify):\n', playlist_df.iloc[null_song_idx]["song_title"])

In [ ]:
features_df = pd.DataFrame()
i = 1
for audio_features in track_audio_features:
    song_features = pd.DataFrame(audio_features, index=[i])
#    print(audio_features.keys())
    features_df = pd.concat([features_df,song_features], axis=0)
    i+=1
features_df


## join the audio features df with the playlist df  
ultimately this might not be necessary because we have the track id and all of the info we want for now. 

In [ ]:
complete_playlist_df = playlist_df.merge(features_df, left_on='track_id', right_on='id', how='left')

In [ ]:
complete_playlist_df = complete_playlist_df[complete_playlist_df['track_id'].notna()]
complete_playlist_df

In [ ]:
complete_playlist_df.columns

## get the count of each artist to find the most popular in the playlist

In [ ]:
#playlist_df['artist_id'].value_counts()

In [ ]:
#playlist_df['artist'].value_counts()

try to get all of the artists individually instead of per track. (tracks with more than 1 artist have a LIST of artists as its value in this column)

In [ ]:
artist_counts_in_playlist = {} #count the occurrences of each artist in the playlist (including artist features)
artist_id_counts_in_playlist = {}

artists = complete_playlist_df['artist']
artist_ids = complete_playlist_df['artist_id']

for i in artists.index:
    
    track_artists_lst = artists[i]
    track_artist_ids_lst = artist_ids[i]
    
    for a in track_artists_lst:
        if a not in artist_counts_in_playlist.keys():
            artist_counts_in_playlist[a] = 1
        else:
            artist_counts_in_playlist[a] += 1
            
    for a_id in track_artist_ids_lst:
        if a_id not in artist_id_counts_in_playlist.keys():
            artist_id_counts_in_playlist[a_id] = 1
        else:
            artist_id_counts_in_playlist[a_id] += 1
        
            
print(artist_counts_in_playlist)
#artist_counts = pd.DataFrame.from_dict(artist_counts_in_playlist, columns=['artist', 'playlist_appearances'])
artist_counts = pd.DataFrame(artist_counts_in_playlist, index=range(0,1))
artist_counts

In [ ]:
artist_id_counts = pd.DataFrame(artist_id_counts_in_playlist, index=range(0,1))
artist_id_counts

In [ ]:
#artist_id_counts.T.sort_values(by=0,ascending=False)

In [ ]:
import seaborn as sns

In [ ]:
# ax = plt.subplots(figsize=(20,10))
# ax = sns.barplot(artist_id_counts, palette="Paired")
# ax = plt.xticks(rotation=75)
# #sorrt the bars

weights could be assigned by frequency. Making an assumption here that if an artist appears more in a playlist its because the user prefers their music. 

## search for tracks by the artist

artist_top_tracks

In [ ]:
artist_id_counts.columns

In [ ]:
artist_id_counts.columns[0]

searching by artist name (easier than searching by id so far. id wouldnt return any results)

In [ ]:
result = sp.search(artist_counts.columns[0], type=['track', 'album'], limit = 50) #limit is 50 because we want to make sure that we're recommending songs not already in a playlist
print('len result[tracks]',len(result['tracks']))
print(result['tracks'])

In [ ]:
print('len result[tracks][items]',len(result['tracks']['items']))
# print(result['tracks']['items'])
#print (json.dumps(result['tracks']['items'], indent=4, default=str))
#print (type(result['tracks']['items']))
song_names = []
song_ids = []

for song in result['tracks']['items']:
    song_names.append(song['name'])
    song_ids.append(song['id'])
    
d = {'song':song_names,
     'song_id':song_ids}
pd.DataFrame(d, index=None)


In [ ]:
song_ids

In [ ]:
song_ids_to_analyze =[s for s in song_ids if s not in complete_playlist_df.track_id ] #the songs from the artist search that are not in the playlist already

In [ ]:
len(song_ids_to_analyze)

make sure that these songs are not on the playlist already  
get the audio features for these songs  
compare to the songs that are already in the playlist:  
* comparing just to the song fom the playlist that the search started with?
* should i calculate some sort of average from the audio features of the songs in the playlist?

Find the average value of the audio features in the playlist

In [ ]:
complete_playlist_df.columns

In [ ]:
playlist_track_audio_features = complete_playlist_df[["track_id", "danceability", "energy", "key",
"loudness", "mode", "speechiness", "acousticness",
"instrumentalness", "liveness", "valence"]]

In [ ]:
playlist_track_audio_features

normalize all the audio features

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
std_num_playlist_track_audio_features = pd.DataFrame(scaler.fit_transform(playlist_track_audio_features.iloc[:,1:]))

In [ ]:
std_num_playlist_track_audio_features = pd.DataFrame(playlist_track_audio_features.iloc[:,0]).join(std_num_playlist_track_audio_features)
std_num_playlist_track_audio_features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
minmax_playlist_track_audio_features = scaler.fit_transform(playlist_track_audio_features.iloc[:,1:])

In [ ]:
minmax_playlist_track_audio_features = pd.DataFrame(minmax_playlist_track_audio_features)

In [ ]:
minmax_playlist_track_audio_features = pd.DataFrame(playlist_track_audio_features.iloc[:,0]).join(minmax_playlist_track_audio_features)
minmax_playlist_track_audio_features

In [ ]:
audio_features_avg_std = std_num_playlist_track_audio_features.mean()

In [ ]:
audio_features_avg_minmax = minmax_playlist_track_audio_features.mean()

In [ ]:
from sklearn.metrics import pairwise

In [ ]:
np.array(std_num_playlist_track_audio_features.iloc[1,1:])

In [ ]:
audio_features_avg_std

Cosine similarity between the average auio feature values of all tracks on playlist and the first track. this calculation is going to be performed for eah track in the playlist to identify which songs on the playlist should be used for recommendation. Those songs that are the most similar to the values of the playlist overall will be put through the search process above.

In [ ]:
std_num_playlist_track_audio_features.iloc[1,1:]

In [ ]:
pairwise.cosine_similarity(np.array(std_num_playlist_track_audio_features.iloc[1,1:]).reshape(1, -1), np.array(audio_features_avg_std).reshape(1, -1))

In [ ]:
pairwise.cosine_similarity(np.array(minmax_playlist_track_audio_features.iloc[1,1:]).reshape(1, -1), np.array(audio_features_avg_minmax).reshape(1, -1))

so far I like the minmax scaled dataset as opposed to the standard scaler dataset because the values seem cleaner

### NO cannot search by features

### if i cant then how would be the best way to recommend people music?
- i can start by looking at the specific artist, searching their song catalog for ones with similar features [songs with completely different features but by the same artist could be suggested here to show artists range]
- find similar artists:
    - spotify API
    - searching through current Artist's features & featured artists
    - search the genre? (dont have this information at this point, but can it be found at a higher level?)
- make sure that there are absolutely NO songs from the original playlist that are on the new one that is generated
    
### another idea is to keep the songs that are seen & their info in a database...  
- this would add overhead that im not sure i want to add right now
- database could essentially grow to MASSIVE sizes, and i dont have MASSIVE resources. 

### incorporate feedback on the playlist somehow? or on the song? idk

### could analyze the new & old playlists as well. Would be cool to display some visualizations based on them
try to incorporate new playlist images somehow if web ui

spotipy is more of a wrapper in the sense that it just provides a simpler way to make calls to the web API. Here is the main code for the client. It just builds a structured HTTP request for you https://github.com/spotipy-dev/spotipy/blob/master/spotipy/client.py#L217-L291.

in order to evaluate performance, can have the application 'finish' playlists & see if there are any songs that are recommended which actually appear on the playlist. otherwise, how could this be evaluated/validated? 

a user profile could THEORETICALLY be built using the playlist, but it is hard to do without explicit rating data.  
what if the app named 10 songs to ask the user their rating (1-5) for>

In [ ]:
sp.audio_features('2m85exTjr6xIRSlhx92MUJ')

In [ ]:
sp.audio_features(['4CYTQpr2jc4uBScYvpEK2w'])